In [209]:
from gamspy import Container
from gamspy import Model
from gamspy import Set
from gamspy import Parameter
from gamspy import Variable
from gamspy import Equation

import pandas as pd
import numpy as np

import os

## DATA DEFINITION

In [210]:
data_path = "data"

# Set i
products_info = pd.read_excel(os.path.join(data_path, "Products.xlsx"), sheet_name="Sayfa1", index_col=None)
products = [i.replace(" ", "") for i in products_info.Products.tolist()]
set_i = products

# Set j
distance = pd.read_excel(os.path.join(data_path, "distance-matrix.xlsx"), sheet_name="Sayfa1", index_col=0).fillna(0)
cities = [i.replace("İ", "i").lower() for i in distance.columns]
distance.columns = a
distance.index = a
set_j = cities

# Set k
ports = pd.read_excel(os.path.join(data_path, "Limanlar.xlsx"), sheet_name="Sayfa1", index_col=None)
ports = [i.replace("İ", "i").lower() for i in ports.PortName.to_list()]
set_k = ports


In [211]:
# Parametes

# w
forecasts = pd.read_csv(os.path.join(data_path, "export_forecasts.csv"), index_col=0)
param_w = np.array(forecasts["2024"])

# d
param_d = distance[ports].unstack()

# l
### TODO: l is not defined

# q
quality = pd.read_excel(os.path.join(data_path, "Quality_ProdCap.xlsx"), sheet_name="Quality", index_col=0).T
quality.columns = [i.replace("İ", "i").lower() for i in quality.columns]
param_q = quality[cities].unstack()

# v
param_v = np.array([int(i.split()[0]) for i in products_info["Quantity (1 Container)"]])

# cap
product_capacity = pd.read_excel(os.path.join(data_path, "Quality_ProdCap.xlsx"), sheet_name="ProdCap", index_col=0).T
product_capacity.columns = [i.replace("İ", "i").lower() for i in product_capacity.columns]
param_cap = product_capacity[cities].unstack()


## BUILD MODELS

In [212]:
# Create a container
m = Container()


### SETS

In [213]:
# Create the model sets
i = Set(m, name='i', description='Products considered', records=set_i)
j = Set(m, name='j', description='Potential warehouse locations', records=set_j)
k = Set(m, name='k', domain=j, description='Exporting Hubs (Ports)', records=set_k)

In [219]:
d.getDomainViolations()

[<DomainViolation (0x2509404ba10)>]

### PARAMETERS

In [215]:
# Create the model parameters
w = Parameter(m, name='w', domain=i, description='Weights for product i', records=param_w)
d = Parameter(m, name='d', domain=[j, k], description='Distance from city j to exporting hub k', records=param_d)
# l = Parameter(m, name='l', domain=j, description='Living expenses at city j')   # ToDO: Get the living expenses from the data --> Ibrahim
q = Parameter(m, name='q', domain=[i, j], description='Quality of product i in city j', records=param_q)
v = Parameter(m, name='v', domain=i, description='Volume of product i', records=param_v)
# av = Parameter(m, name='av', domain=j, description='Available labor at city j') # ToDO: Get the available labor from the data --> Ibrahim
cap = Parameter(m, name='cap', domain=[i, j], description='Production Capacity of product i in city j', records=param_cap)


### VARIABLES

In [216]:
# Create the model variables
x = Variable(m, name='x', domain=j, type="binary", description='Whether to build a warehouse in city j')
y = Variable(m, name='y', domain=[i, j], type="positive", description='Amount of product i stocked at city j')


In [217]:
d.records

,j,k,value
0,antalya,adana,547.0
1,antalya,ankara,542.0
2,antalya,antalya,0.0
3,antalya,aydin,345.0
4,antalya,balikesir,505.0
...,...,...,...
265,trabzon,samsun,321.0
266,trabzon,tekirdağ,1187.0
267,trabzon,trabzon,0.0
268,trabzon,şanliurfa,804.0
